In [2]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from datetime import date as d, timedelta as dt
from newspaper import Article, Config

In [5]:
# may have result pages less than 2
# google links that contains 10 news per page
search_urls = {}

# search a year, 2 pages per day
for dr in range(365):
    date = (d.today() - dt(dr)).strftime('%m/%d/%Y')
    search_urls[f'{date}'] = [f'https://www.google.com/search?q=stock+market&source=lnt&tbs=cdr%3A1%2Ccd_min%3A{date}%2Ccd_max%3A{date}&tbm=nws&start={page*10}' for page in range(2)]



In [60]:
from tqdm import tqdm

In [15]:
header = {"User-Agent": "Mozilla/5.0 (X11; CrOS x86_64 12871.102.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.141 Safari/537.36"}
news_urls = {}

# get all news links by date
for date, urls in tqdm(search_urls.items()):
    page = [requests.get(url, headers=header) for url in urls]
    soup = [BeautifulSoup(p.content, 'html.parser') for p in page]
    main = [i.find_all('a', class_='WlydOe') for i in soup]

    all_in_a_day = []
    
    for i in main:
        all_in_a_day += [j['href'] for j in i]
    news_urls[f'{date}'] = all_in_a_day


100%|██████████| 365/365 [06:40<00:00,  1.10s/it]


In [16]:
config = Config()
config.MAX_SUMMARY_SENT = 10

dates = []
urls = []
full_text = []
summary = []

# get summary and full text from each news link
for date, news_urls in tqdm(news_urls.items()):
    for url in news_urls:
        try:
            article = Article(url, config=config)
            article.download()
            article.parse()
            article.nlp()
            s = article.summary
            if len(s) >= 500:
                dates.append(date)
                urls.append(url)
                full_text.append(article.text)
                summary.append(s)
        except:
            continue



  7%|▋         | 24/365 [08:41<2:21:16, 24.86s/it]c:\Users\kian\anaconda3\lib\site-packages\dateutil\parser\_parser.py:1207: UnknownTimezoneWarning: tzname PST identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  warnings.warn("tzname {tzname} identified but not understood.  "
  9%|▉         | 33/365 [12:32<2:07:31, 23.05s/it]c:\Users\kian\anaconda3\lib\site-packages\dateutil\parser\_parser.py:1207: UnknownTimezoneWarning: tzname CDT identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  warnings.warn("tzname {tzname} identified but not understood.  "
 59%|█████▉    | 215/365 [1:10:05<51:12, 20.48s/it]  c:\Users\kian\anaconda3\lib\site-packages\dateutil\parser\_parser.py:1207: UnknownTimezoneWarning: tzname CST identified but not understood.  Pass `tzinfos` argument

In [17]:
k = pd.DataFrame({'date': dates, 'url': urls, 'full_text': full_text, 'summary': summary})

In [18]:
k

,date,url,full_text,summary
0,10/08/2022,https://www.livemint.com/market/stock-market-n...,Have you read your account statements for the ...,And it’s natural to avoid looking too closely ...
1,10/08/2022,https://www.zawya.com/en/markets/equities/wall...,Wall Street fell sharply on Friday following a...,"Nonfarm payrolls rose by 263,000 jobs, more th..."
2,10/08/2022,https://www.goodisonnews.com/2022/10/08/stock-...,"By Matthew Shaw 8th Oct, 2022 | 8:10am\n\nStoc...","By Matthew Shaw 8th Oct, 2022 | 8:10amStock ma..."
3,10/08/2022,https://www.livemint.com/market/stock-market-n...,Stock split 2022: The board of directors of An...,Stock split 2022: The board of directors of An...
4,10/08/2022,https://finance.yahoo.com/news/crypto-exchange...,(Reuters) - Cryptocurrency exchange Huobi Glob...,(Reuters) - Cryptocurrency exchange Huobi Glob...
...,...,...,...,...
4052,12/01/2021,https://www.aarp.org/money/investing/info-2021...,Matthew Salacuse\n\nEn español\n\nFor 16 years...,"Matthew SalacuseEn españolFor 16 years, the ex..."
4053,12/01/2021,https://fortune.com/2021/12/01/stock-market-pu...,"Dante had nine circles of hell in his Inferno,...","Dante had nine circles of hell in his Inferno,..."
4054,12/01/2021,https://www.fool.com/investing/2021/12/01/3-re...,Pfizer (PFE -1.79%) has been the second-best-p...,Pfizer (PFE -1.79%) has been the second-best-p...
4055,12/01/2021,https://www.globenewswire.com/news-release/202...,Global leader in Connected TV ad delivery and ...,"According to eMarketer, CTV ad spend will reac..."


### Improvement
- the script runs for 2 hours to scrape 365 days of data
- there are only 312 unique dates
- different numbers of unique url, summary, full_text, meaning some replicated news in different websites
- or the requests were blocked by paywall, reporting same text
- raw text is not cleaned

#### Undate about the dates
- found that dates only contained up to Dec 2021
- the requests are block by google (response 429)

### Try to search again on dates that are missing after unblock

In [3]:
saved = pd.read_csv('news.csv')
saved_dates = saved.date.unique()

In [6]:
all_dates = [key for key, v in search_urls.items()]
date_missing = [i for i in all_dates if i not in saved_dates]

In [56]:
urls_missing = {}

for day in date_missing:
    urls_missing[f'{day}'] = search_urls[f'{day}']

In [61]:
header = {"User-Agent": "Mozilla/5.0 (X11; CrOS x86_64 12871.102.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.141 Safari/537.36"}
news_urls = {}


for date, urls in tqdm(urls_missing.items()):
    page = [requests.get(url, headers=header) for url in urls]
    soup = [BeautifulSoup(p.content, 'html.parser') for p in page]
    main = [i.find_all('a', class_='WlydOe') for i in soup]

    all_in_a_day = []
    
    for i in main:
        all_in_a_day += [j['href'] for j in i]
    news_urls[f'{date}'] = all_in_a_day

100%|██████████| 53/53 [00:54<00:00,  1.02s/it]


In [63]:
config = Config()
config.MAX_SUMMARY_SENT = 10

dates = []
urls = []
full_text = []
summary = []

for date, news_urls in tqdm(news_urls.items()):
    for url in news_urls:
        try:
            article = Article(url, config=config)
            article.download()
            article.parse()
            article.nlp()
            s = article.summary
            if len(s) >= 500:
                dates.append(date)
                urls.append(url)
                full_text.append(article.text)
                summary.append(s)
        except:
            continue

 13%|█▎        | 7/53 [02:22<16:20, 21.31s/it]c:\Users\kian\anaconda3\lib\site-packages\dateutil\parser\_parser.py:1207: UnknownTimezoneWarning: tzname PST identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  warnings.warn("tzname {tzname} identified but not understood.  "
100%|██████████| 53/53 [16:49<00:00, 19.04s/it]


In [64]:
news_missing = pd.DataFrame({'date': dates, 'url': urls, 'full_text': full_text, 'summary': summary})

In [71]:
new_save = pd.concat([saved, news_missing]).iloc[:, 2:]

new_save.to_csv('news.csv')

In [72]:
new_save

,date,url,full_text,summary
0,10/08/2022,https://www.livemint.com/market/stock-market-n...,Have you read your account statements for the ...,And it’s natural to avoid looking too closely ...
1,10/08/2022,https://www.zawya.com/en/markets/equities/wall...,Wall Street fell sharply on Friday following a...,"Nonfarm payrolls rose by 263,000 jobs, more th..."
2,10/08/2022,https://www.goodisonnews.com/2022/10/08/stock-...,"By Matthew Shaw 8th Oct, 2022 | 8:10am\n\nStoc...","By Matthew Shaw 8th Oct, 2022 | 8:10amStock ma..."
3,10/08/2022,https://www.livemint.com/market/stock-market-n...,Stock split 2022: The board of directors of An...,Stock split 2022: The board of directors of An...
4,10/08/2022,https://finance.yahoo.com/news/crypto-exchange...,(Reuters) - Cryptocurrency exchange Huobi Glob...,(Reuters) - Cryptocurrency exchange Huobi Glob...
...,...,...,...,...
637,10/09/2021,https://www.fool.com/investing/2021/10/09/shou...,"After an incredible 18 months of growth, the s...","After an incredible 18 months of growth, the s..."
638,10/09/2021,https://www.fool.com/investing/2021/10/09/when...,It's an immutable fact that the stock market i...,It's an immutable fact that the stock market i...
639,10/09/2021,https://abcnews.go.com/Business/stocks-soar-hi...,One overvaluation indicator is higher than it ...,The stock market has been a roller coaster rid...
640,10/09/2021,https://www.fool.com/investing/2021/10/09/this...,Real estate tech company and iBuyer Opendoor T...,This move could mean that the race to change t...


### Get SP500, Nasdaq, DJIA by date 